# Interest Rate Predictor CaseStudy

In this case study, we will be using Linear Regression.

A little background on the case study. This data belongs to a loan aggregator agency which connects loan applications to different financial institutions in attempt to get the best interest rate. They want to now utilise past data to predict interest rate given by any financial institute just by looking at loan application characteristics.

To achieve that , they have decided to do a POC with a data from a particular financial institution.

In [73]:
# Importing libraries
import numpy as np
import pandas as pd

In [74]:
# Reading Train and Test dataset and viewing size
train_file = "Data/loan_data_train.csv"
test_file = "Data/loan_data_test.csv"

ld_train = pd.read_csv(train_file)
ld_test = pd.read_csv(test_file)
print(ld_train.shape)
ld_train.head()

(2200, 15)


,ID,Amount.Requested,Amount.Funded.By.Investors,Interest.Rate,Loan.Length,Loan.Purpose,Debt.To.Income.Ratio,State,Home.Ownership,Monthly.Income,FICO.Range,Open.CREDIT.Lines,Revolving.CREDIT.Balance,Inquiries.in.the.Last.6.Months,Employment.Length
0,79542.0,25000,25000,18.49%,60 months,debt_consolidation,27.56%,VA,MORTGAGE,8606.56,720-724,11,15210,3.0,5 years
1,75473.0,19750,19750,17.27%,60 months,debt_consolidation,13.39%,NY,MORTGAGE,6737.50,710-714,14,19070,3.0,4 years
2,67265.0,2100,2100,14.33%,36 months,major_purchase,3.50%,LA,OWN,1000.00,690-694,13,893,1.0,< 1 year
3,80167.0,28000,28000,16.29%,36 months,credit_card,19.62%,NV,MORTGAGE,7083.33,710-714,12,38194,1.0,10+ years
4,17240.0,24250,17431.82,12.23%,60 months,credit_card,23.79%,OH,MORTGAGE,5833.33,730-734,6,31061,2.0,10+ years


In [75]:
#test data does not have interest rate
print(ld_test.shape)
ld_test.head()

(300, 14)


,ID,Amount.Requested,Amount.Funded.By.Investors,Loan.Length,Loan.Purpose,Debt.To.Income.Ratio,State,Home.Ownership,Monthly.Income,FICO.Range,Open.CREDIT.Lines,Revolving.CREDIT.Balance,Inquiries.in.the.Last.6.Months,Employment.Length
0,20093,5000,5000,60 months,moving,12.59%,NY,RENT,4416.67,690-694,13,7686,0,< 1 year
1,62445,18000,18000,60 months,debt_consolidation,4.93%,CA,RENT,5258.50,710-714,6,11596,0,10+ years
2,65248,7200,7200,60 months,debt_consolidation,25.16%,LA,MORTGAGE,3750.00,750-754,13,7283,0,6 years
3,81822,7200,7200,36 months,debt_consolidation,17.27%,NY,MORTGAGE,3416.67,790-794,14,4838,0,10+ years
4,57923,22000,22000,60 months,debt_consolidation,18.28%,MI,MORTGAGE,6083.33,720-724,9,20181,0,8 years


In [76]:
# Combine Test and Train dataset to perform data cleaning
ld_test['Interest.Rate'] = np.nan
ld_train['data'] = 'train'
ld_test['data'] = 'test'
ld_test = ld_test[ld_train.columns]                  # ld_test columns should be in same sequence as ld_train
ld_all = pd.concat([ld_train,ld_test],axis=0)

In [77]:
print(ld_all.shape)
ld_all.head()

(2500, 16)


,ID,Amount.Requested,Amount.Funded.By.Investors,Interest.Rate,Loan.Length,Loan.Purpose,Debt.To.Income.Ratio,State,Home.Ownership,Monthly.Income,FICO.Range,Open.CREDIT.Lines,Revolving.CREDIT.Balance,Inquiries.in.the.Last.6.Months,Employment.Length,data
0,79542.0,25000,25000,18.49%,60 months,debt_consolidation,27.56%,VA,MORTGAGE,8606.56,720-724,11,15210,3.0,5 years,train
1,75473.0,19750,19750,17.27%,60 months,debt_consolidation,13.39%,NY,MORTGAGE,6737.50,710-714,14,19070,3.0,4 years,train
2,67265.0,2100,2100,14.33%,36 months,major_purchase,3.50%,LA,OWN,1000.00,690-694,13,893,1.0,< 1 year,train
3,80167.0,28000,28000,16.29%,36 months,credit_card,19.62%,NV,MORTGAGE,7083.33,710-714,12,38194,1.0,10+ years,train
4,17240.0,24250,17431.82,12.23%,60 months,credit_card,23.79%,OH,MORTGAGE,5833.33,730-734,6,31061,2.0,10+ years,train


In [78]:
# Checking Data Types for all columns
ld_all.dtypes

ID                                float64
Amount.Requested                   object
Amount.Funded.By.Investors         object
Interest.Rate                      object
Loan.Length                        object
Loan.Purpose                       object
Debt.To.Income.Ratio               object
State                              object
Home.Ownership                     object
Monthly.Income                    float64
FICO.Range                         object
Open.CREDIT.Lines                  object
Revolving.CREDIT.Balance           object
Inquiries.in.the.Last.6.Months    float64
Employment.Length                  object
data                               object
dtype: object

## Observations

1. ID,Amount.Funded.By.Investors columns can be drop as they don't have any impact on Interest Rate
2. For Columns Interest Rate , Debt to income ratio : remove % and then convert to numeric
3. Amount.Requested , 'Open.CREDIT.Lines','Revolving.CREDIT.Balance': convert it to numeric 
4. FICO.Range : replace it by a numeric column which is average of the range
5. Employment Length : convert to number
6. Loan Lenth, Loan Purpose , State , Home ownership: create dummies for categories with good occurence rate

In [79]:
# Dropping columns which don't have any impact on Target Variable 
ld_all = ld_all.drop(['ID','Amount.Funded.By.Investors'],axis=1)

In [80]:
# Converting Interest.Rate and Debt.To.Income.Ratio to numeric
for col in ['Interest.Rate','Debt.To.Income.Ratio']:
    ld_all[col] = ld_all[col].str.replace("%","")
ld_all.head()

,Amount.Requested,Interest.Rate,Loan.Length,Loan.Purpose,Debt.To.Income.Ratio,State,Home.Ownership,Monthly.Income,FICO.Range,Open.CREDIT.Lines,Revolving.CREDIT.Balance,Inquiries.in.the.Last.6.Months,Employment.Length,data
0,25000,18.49,60 months,debt_consolidation,27.56,VA,MORTGAGE,8606.56,720-724,11,15210,3.0,5 years,train
1,19750,17.27,60 months,debt_consolidation,13.39,NY,MORTGAGE,6737.50,710-714,14,19070,3.0,4 years,train
2,2100,14.33,36 months,major_purchase,3.50,LA,OWN,1000.00,690-694,13,893,1.0,< 1 year,train
3,28000,16.29,36 months,credit_card,19.62,NV,MORTGAGE,7083.33,710-714,12,38194,1.0,10+ years,train
4,24250,12.23,60 months,credit_card,23.79,OH,MORTGAGE,5833.33,730-734,6,31061,2.0,10+ years,train


In [81]:
# Converting columns to Numeric
for col in ['Amount.Requested','Interest.Rate','Debt.To.Income.Ratio','Open.CREDIT.Lines','Revolving.CREDIT.Balance']:
    ld_all[col] = pd.to_numeric(ld_all[col],errors='coerce')
ld_all.dtypes

Amount.Requested                  float64
Interest.Rate                     float64
Loan.Length                        object
Loan.Purpose                       object
Debt.To.Income.Ratio              float64
State                              object
Home.Ownership                     object
Monthly.Income                    float64
FICO.Range                         object
Open.CREDIT.Lines                 float64
Revolving.CREDIT.Balance          float64
Inquiries.in.the.Last.6.Months    float64
Employment.Length                  object
data                               object
dtype: object

In [82]:
# Handling FICO.Range Column
ld_all['FICO1'] = [value[0:3] for value in ld_all['FICO.Range']]
ld_all['FICO2'] = [value[4:7] for value in ld_all['FICO.Range']]
ld_all['FICO'] = 0.5 * (ld_all['FICO1'].astype(float) + ld_all['FICO2'].astype(float))
ld_all = ld_all.drop(['FICO1','FICO2','FICO.Range'],axis=1)

In [83]:
ld_all.head()

,Amount.Requested,Interest.Rate,Loan.Length,Loan.Purpose,Debt.To.Income.Ratio,State,Home.Ownership,Monthly.Income,Open.CREDIT.Lines,Revolving.CREDIT.Balance,Inquiries.in.the.Last.6.Months,Employment.Length,data,FICO
0,25000.0,18.49,60 months,debt_consolidation,27.56,VA,MORTGAGE,8606.56,11.0,15210.0,3.0,5 years,train,722.0
1,19750.0,17.27,60 months,debt_consolidation,13.39,NY,MORTGAGE,6737.50,14.0,19070.0,3.0,4 years,train,712.0
2,2100.0,14.33,36 months,major_purchase,3.50,LA,OWN,1000.00,13.0,893.0,1.0,< 1 year,train,692.0
3,28000.0,16.29,36 months,credit_card,19.62,NV,MORTGAGE,7083.33,12.0,38194.0,1.0,10+ years,train,712.0
4,24250.0,12.23,60 months,credit_card,23.79,OH,MORTGAGE,5833.33,6.0,31061.0,2.0,10+ years,train,732.0


In [84]:
# Converting Employment.Length column to numeric
ld_all['Employment.Length'] = ld_all['Employment.Length'].str.replace(" years","")
ld_all['Employment.Length'] = ld_all['Employment.Length'].str.replace(" year","")
ld_all['Employment.Length'] = ld_all['Employment.Length'].str.replace("< ","")
ld_all['Employment.Length'] = ld_all['Employment.Length'].str.replace("+","")

In [85]:
ld_all['Employment.Length'] = pd.to_numeric(ld_all['Employment.Length'],errors='coerce')

In [86]:
# Handling Categorical Columns
cat_col = ld_all.select_dtypes(['object']).columns
cat_col = cat_col[:-1]
cat_col

Index(['Loan.Length', 'Loan.Purpose', 'State', 'Home.Ownership'], dtype='object')

In [87]:
# We can use following method if we want to ignore categories with too low frequencies ,
# also we can use get dummies function. 
# we can work with either of these . 
#ignoring categories with low frequencies however will result in fewer columns without 
# affecting model performance too much .

In [88]:
for col in cat_col:
    freqs = ld_all[col].value_counts()
    k = freqs.index[freqs>20][:-1]
    for cat in k:
        name = col + "_" + cat
        ld_all[name] = (ld_all[col]==cat).astype(int)
    del ld_all[col]
    print(col)

Loan.Length
Loan.Purpose
State
Home.Ownership


In [89]:
ld_all.shape

(2500, 51)

In [90]:
# Handling NA Values
print(ld_all.isnull().sum())
for col in ld_all.columns:
    if (col not in ['Interest.Rate','data']) & (ld_all[col].isnull().sum()>0):
        ld_all.loc[ld_all[col].isnull(),col] = ld_all.loc[ld_all['data'] == 'train',col].mean()

Amount.Requested                     5
Interest.Rate                      300
Debt.To.Income.Ratio                 1
Monthly.Income                       3
Open.CREDIT.Lines                    9
Revolving.CREDIT.Balance             5
Inquiries.in.the.Last.6.Months       3
Employment.Length                   80
data                                 0
FICO                                 0
Loan.Length_36 months                0
Loan.Purpose_debt_consolidation      0
Loan.Purpose_credit_card             0
Loan.Purpose_other                   0
Loan.Purpose_home_improvement        0
Loan.Purpose_major_purchase          0
Loan.Purpose_small_business          0
Loan.Purpose_car                     0
Loan.Purpose_wedding                 0
Loan.Purpose_medical                 0
Loan.Purpose_moving                  0
State_CA                             0
State_NY                             0
State_TX                             0
State_FL                             0
State_IL                 

In [91]:
ld_all.isnull().sum()

Amount.Requested                     0
Interest.Rate                      300
Debt.To.Income.Ratio                 0
Monthly.Income                       0
Open.CREDIT.Lines                    0
Revolving.CREDIT.Balance             0
Inquiries.in.the.Last.6.Months       0
Employment.Length                    0
data                                 0
FICO                                 0
Loan.Length_36 months                0
Loan.Purpose_debt_consolidation      0
Loan.Purpose_credit_card             0
Loan.Purpose_other                   0
Loan.Purpose_home_improvement        0
Loan.Purpose_major_purchase          0
Loan.Purpose_small_business          0
Loan.Purpose_car                     0
Loan.Purpose_wedding                 0
Loan.Purpose_medical                 0
Loan.Purpose_moving                  0
State_CA                             0
State_NY                             0
State_TX                             0
State_FL                             0
State_IL                 

- Data Cleaning Process is Completed
- Now we can seperate Test and Train Data Sets.
- Using Cross Validation Technique we will divide Train Data sets in 2 parts, build model and validate

In [92]:
ld_train = ld_all[ld_all['data']=='train']
ld_train = ld_train.drop(['data'],axis=1)

ld_test = ld_all[ld_all['data']=='test']
ld_test = ld_test.drop(['Interest.Rate','data'],axis=1)

del ld_all

In [94]:
from sklearn.model_selection import train_test_split
ld_train1, ld_train2 = train_test_split(ld_train,test_size=0.3,random_state=1)

In [95]:
x_train1 = ld_train1.drop(['Interest.Rate'],axis=1)
y_train1 = ld_train1['Interest.Rate']

In [96]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(x_train1,y_train1)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [97]:
lr.intercept_

76.70196805637175

In [98]:
list(zip(x_train1.columns,lr.coef_))

[('Amount.Requested', 0.0001551390963903276),
 ('Debt.To.Income.Ratio', -0.00024157406448616986),
 ('Monthly.Income', -1.7536246891700094e-05),
 ('Open.CREDIT.Lines', -0.02851362087438844),
 ('Revolving.CREDIT.Balance', -3.158125018998259e-06),
 ('Inquiries.in.the.Last.6.Months', 0.2965940672105093),
 ('Employment.Length', 0.0365576493134574),
 ('FICO', -0.08802552052324819),
 ('Loan.Length_36 months', -3.1838678472176456),
 ('Loan.Purpose_debt_consolidation', -0.5645696703995263),
 ('Loan.Purpose_credit_card', -0.45950640621108674),
 ('Loan.Purpose_other', 0.43105339118217634),
 ('Loan.Purpose_home_improvement', -0.7759036800153263),
 ('Loan.Purpose_major_purchase', -0.24985553576409386),
 ('Loan.Purpose_small_business', 0.0038121370566041035),
 ('Loan.Purpose_car', -0.46247414178681256),
 ('Loan.Purpose_wedding', -0.4209707169635378),
 ('Loan.Purpose_medical', -0.4269272792837086),
 ('Loan.Purpose_moving', 0.9897346066005912),
 ('State_CA', -0.3441837022144685),
 ('State_NY', -0.2728

In [100]:
x_train2 = ld_train2.drop(['Interest.Rate'],axis=1)
predictedInterestRate = lr.predict(x_train2)

In [101]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(predictedInterestRate,ld_train2['Interest.Rate'])

1.6279117116118145

In [102]:
# Now we will apply same model on whole Test Data set and predict Result
x_train = ld_train.drop(['Interest.Rate'],axis=1)
y_train = ld_train['Interest.Rate']
lr.fit(x_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [103]:
predictedIR_test = lr.predict(ld_test)

In [104]:
pd.DataFrame(predictedIR_test).to_csv("LinearRegression.csv",index=False)